In [1]:
# get openai api key from openai_config.json file
import json
with open('openai_config.json') as f:
    config =json.load(f)
    openai_api_key = config['openai_api_key']
    NCP_CLOVASTUDIO_API_KEY = config['NCP_CLOVASTUDIO_API_KEY']
    NCP_APIGW_API_KEY = config['NCP_APIGW_API_KEY']
    NCP_CLOVASTUDIO_APP_ID = config['NCP_CLOVASTUDIO_APP_ID']

In [ ]:
%env OPENAI_API_KEY = {openai_api_key}
%env NCP_CLOVASTUDIO_API_KEY = {NCP_CLOVASTUDIO_API_KEY}
%env NCP_APIGW_API_KEY = {NCP_APIGW_API_KEY}
%env NCP_CLOVASTUDIO_APP_ID = {NCP_CLOVASTUDIO_APP_ID}

In [3]:
from langchain_community.chat_models import ChatClovaX
chat = ChatClovaX(
    model="HCX-003",
    max_tokens=1000,
    temperature=0.5,)

In [4]:
novel_prompt = """너는 단문 소설을 쓰는 작가야. 주어진 단어를 포함하는 세 문단짜리 단문 소설을 작문해야 해.
아래 단어를 모두 포함하도록 단문 소설을 써 줘.
{keywords}
제목 없이 소설 내용만 써 줘. 주어진 단어를 활용하는 자연스러운 전개로 써 줘.
전지적 작가 시점으로 써 줘.
인물의 대사 한 개 이상을 포함해.
소설은 세 문단으로 이루어져야 해. 세 문단 이상 쓰지 마.
"""
science_prompt = """너는 과학적 사실을 설명하는 글을 쓰는 과학 저널리스트야. 주어진 과학 질문에 대한 답변을 세 문단으로 작성해야 해.
아래 과학 질문에 대한 답변을 써 줘.
{question}
제목이나 요약 없이 답변 내용만 써 줘. 질문과 관련없는 내용은 쓰지 마.
-다, 와 같은 종결어미를 사용하는 해라체로 작성해.
답변은 세 문단으로 이루어져야 해.
"""

In [20]:
import csv

results = []
with open('hw4_data/sample_novel.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        keywords = row
        novel_messages = [
            (
                "system",
                "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.",
            ),
            ("human", novel_prompt.format(keywords=row)),
        ]
        ai_msg = chat.invoke(novel_messages)
        results.append((keywords, ai_msg.content))

In [22]:
with open('hw4_results/sample_novel_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for res in results:
        writer.writerow(res)

In [6]:
import csv

results = []
with open('hw4_data/sample_science.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        keywords = row
        novel_messages = [
            (
                "system",
                "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.",
            ),
            ("human", science_prompt.format(question=row)),
        ]
        ai_msg = chat.invoke(novel_messages)
        results.append((keywords, ai_msg.content))

In [7]:
with open('hw4_results/clova_sample_science_results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for res in results:
        writer.writerow(res)